In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pandas_datareader import data
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV

In [5]:
aapl = data.DataReader("AAPL", 
                       start='2017-1-1', 
                       end='2021-1-1', 
                       data_source='yahoo')
aapl.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-03,29.082500,28.690001,28.950001,29.037500,115127600.0,27.372362
2017-01-04,29.127501,28.937500,28.962500,29.004999,84472400.0,27.341724
2017-01-05,29.215000,28.952499,28.980000,29.152500,88774400.0,27.480770
2017-01-06,29.540001,29.117500,29.195000,29.477501,127007600.0,27.787130
2017-01-09,29.857500,29.485001,29.487499,29.747499,134247600.0,28.041647


In [6]:
df = aapl.reset_index()
df

,Date,High,Low,Open,Close,Volume,Adj Close
0,2017-01-03,29.082500,28.690001,28.950001,29.037500,115127600.0,27.372362
1,2017-01-04,29.127501,28.937500,28.962500,29.004999,84472400.0,27.341724
2,2017-01-05,29.215000,28.952499,28.980000,29.152500,88774400.0,27.480770
3,2017-01-06,29.540001,29.117500,29.195000,29.477501,127007600.0,27.787130
4,2017-01-09,29.857500,29.485001,29.487499,29.747499,134247600.0,28.041647
...,...,...,...,...,...,...,...
1002,2020-12-24,133.460007,131.100006,131.320007,131.970001,54930100.0,131.352829
1003,2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,136.050766
1004,2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,134.239273
1005,2020-12-30,135.990005,133.399994,135.580002,133.720001,96452100.0,133.094650


In [7]:
def create_lag_feature(df, orig_feature, lag, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    # for name, group in new_df.groupby(groupby):        
    if lag < 0: # take values from the past
        new_df.iloc[-lag:][new_feature_name] = df.iloc[:lag][orig_feature].values
    else:       # take values from the future
        new_df.iloc[:-lag][new_feature_name] = df.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

In [9]:
df_hastarget = create_lag_feature(df, 'Adj Close', +28, 'AdjCloseNextMonth', clip=True)
df_hastarget

<ipython-input-7-99e37718b1ae>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[:-lag][new_feature_name] = df.iloc[lag:][orig_feature].values


,Date,High,Low,Open,Close,Volume,Adj Close,AdjCloseNextMonth
0,2017-01-03,29.082500,28.690001,28.950001,29.037500,115127600.0,27.372362,31.547823
1,2017-01-04,29.127501,28.937500,28.962500,29.004999,84472400.0,27.341724,31.957304
2,2017-01-05,29.215000,28.952499,28.980000,29.152500,88774400.0,27.480770,32.073269
3,2017-01-06,29.540001,29.117500,29.195000,29.477501,127007600.0,27.787130,32.035400
4,2017-01-09,29.857500,29.485001,29.487499,29.747499,134247600.0,28.041647,32.122974
...,...,...,...,...,...,...,...,...
974,2020-11-13,119.669998,117.870003,119.440002,119.260002,81581900.0,118.702278,131.352829
975,2020-11-16,120.989998,118.150002,118.919998,120.300003,91183000.0,119.737419,136.050766
976,2020-11-17,120.669998,118.959999,119.550003,119.389999,74271000.0,118.831673,134.239273
977,2020-11-18,119.820000,118.000000,118.610001,118.029999,76322100.0,117.478035,133.094650


In [10]:
split_date = '2020-01-01'
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

In [11]:
preds = df_train['Adj Close']
y_train = df_train['AdjCloseNextMonth']
r2_score(y_train, preds)

0.7922298446537561

In [12]:
preds = df_test['Adj Close']
y_test = df_test['AdjCloseNextMonth']
r2_score(y_test, preds)

0.6475918545585289

In [13]:
#partial code for 1(e) are From Lecture 17
def preprocess_features_num(df_train, df_test, numeric_features, categorical_features, drop_features):

    all_features = set(numeric_features + categorical_features + drop_features)
    if set(df_train.columns) != all_features:
        print("Missing columns", set(df_train.columns) - all_features)
        print("Extra columns", all_features - set(df_train.columns))
        raise Exception("Columns do not match")
    
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])  
    # categorical_transformer = Pipeline([
    #     ('imputer', SimpleImputer(strategy='constant', fill_value='?')),
    #     ('onehot', OneHotEncoder(handle_unknown='ignore'))
    # ])
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, numeric_features),
        # ('categorical', categorical_transformer, categorical_features)
    ])
    preprocessor.fit(df_train)

    # ohe = preprocessor.named_transformers_['categorical'].named_steps['onehot']
    # ohe_feature_names = list(ohe.get_feature_names(categorical_features))
    new_columns = numeric_features
    #  + ohe_feature_names

    X_train_enc = pd.DataFrame(preprocessor.transform(df_train), index=df_train.index, columns=new_columns)
    X_test_enc  = pd.DataFrame(preprocessor.transform(df_test),  index=df_test.index,  columns=new_columns)
    
    y_train = df_train["AdjCloseNextMonth"]
    y_test  = df_test["AdjCloseNextMonth"]
    
    return X_train_enc, y_train, X_test_enc, y_test, preprocessor 


In [14]:
#partial code for 1(e) are From Lecture 17
def preprocess_features(df_train, df_test, numeric_features, categorical_features, drop_features):

    all_features = set(numeric_features + categorical_features + drop_features)
    if set(df_train.columns) != all_features:
        print("Missing columns", set(df_train.columns) - all_features)
        print("Extra columns", all_features - set(df_train.columns))
        raise Exception("Columns do not match")
    
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])  
    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ])
    preprocessor.fit(df_train)

    ohe = preprocessor.named_transformers_['categorical'].named_steps['onehot']
    ohe_feature_names = list(ohe.get_feature_names(categorical_features))
    new_columns = numeric_features + ohe_feature_names

    X_train_enc = pd.DataFrame(preprocessor.transform(df_train), index=df_train.index, columns=new_columns)
    X_test_enc  = pd.DataFrame(preprocessor.transform(df_test),  index=df_test.index,  columns=new_columns)
    
    y_train = df_train["AdjCloseNextMonth"]
    y_test  = df_test["AdjCloseNextMonth"]
    
    return X_train_enc, y_train, X_test_enc, y_test, preprocessor


In [15]:
numeric_features = ['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close']
categorical_features = []
drop_features = ['Date','AdjCloseNextMonth']

In [16]:
df_train.columns[1:-1]

Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [17]:

# Method one: No Trend, No seasonality, no lag
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features_num(df_train, df_test, 
        numeric_features, categorical_features, drop_features)
lr_pipe = make_pipeline(preprocessor, Ridge())
lr_pipe.fit(df_train, y_train)
print('Training Score:',lr_pipe.score(df_train, y_train))
print('Test Score:',lr_pipe.score(df_test, y_test))

Training Score: 0.8452265380555766
Test Score: 0.6967887418213946


In [18]:
df_train = df_train.assign(Month_str=df_train["Date"].apply(lambda x: x.month_name())) # x.month_name() to get the actual string
df_test  = df_test.assign( Month_str=df_test[ "Date"].apply(lambda x: x.month_name()))
df_train = df_train.assign(Month=df_train["Date"].apply(lambda x: x.month)) # x.month_name() to get the actual string
df_test  = df_test.assign( Month=df_test[ "Date"].apply(lambda x: x.month))
df_train = df_train.assign(Month_sin = np.sin(2*np.pi*df_train["Month"]/12))
df_train = df_train.assign(Month_cos = np.cos(2*np.pi*df_train["Month"]/12))
df_test = df_test.assign(Month_sin = np.sin(2*np.pi*df_test["Month"]/12))
df_test = df_test.assign(Month_cos = np.cos(2*np.pi*df_test["Month"]/12))


In [19]:
df_test

,Date,High,Low,Open,Close,Volume,Adj Close,AdjCloseNextMonth,Month_str,Month,Month_sin,Month_cos
754,2020-01-02,75.150002,73.797501,74.059998,75.087502,135480400.0,74.096443,80.911942,January,1,0.5,0.866025
755,2020-01-03,75.144997,74.125000,74.287498,74.357498,146322800.0,73.376083,80.335754,January,1,0.5,0.866025
756,2020-01-06,74.989998,73.187500,73.447502,74.949997,118387200.0,73.960770,80.355545,January,1,0.5,0.866025
757,2020-01-07,75.224998,74.370003,74.959999,74.597504,108872000.0,73.612930,78.884193,January,1,0.5,0.866025
758,2020-01-08,76.110001,74.290001,74.290001,75.797501,132079200.0,74.797081,80.026649,January,1,0.5,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
974,2020-11-13,119.669998,117.870003,119.440002,119.260002,81581900.0,118.702278,131.352829,November,11,-0.5,0.866025
975,2020-11-16,120.989998,118.150002,118.919998,120.300003,91183000.0,119.737419,136.050766,November,11,-0.5,0.866025
976,2020-11-17,120.669998,118.959999,119.550003,119.389999,74271000.0,118.831673,134.239273,November,11,-0.5,0.866025
977,2020-11-18,119.820000,118.000000,118.610001,118.029999,76322100.0,117.478035,133.094650,November,11,-0.5,0.866025


In [24]:
df_train_lag = df_train
df_test_lag = df_test
lag_col = []
df_test_lag
for i in range(1,7):
    df_train_lag = create_lag_feature(df_train_lag, "Adj Close", -i, "AdjClosePrev"+str(i), clip=True)
    lag_col.append("AdjClosePrev"+str(i))
    df_test_lag = create_lag_feature(df_test_lag, "Adj Close", -i, "AdjClosePrev"+str(i), clip=False)

first_day = df_train_lag['Date'].min()
df_train_lag = df_train_lag.assign(Days_since = df_train_lag['Date'].apply(lambda x: (x-first_day).days))
df_test_lag = df_test_lag.assign(Days_since = df_test_lag['Date'].apply(lambda x: (x-first_day).days))

<ipython-input-7-99e37718b1ae>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[-lag:][new_feature_name] = df.iloc[:lag][orig_feature].values


In [25]:
df_test_lag

,Date,High,Low,Open,Close,Volume,Adj Close,AdjCloseNextMonth,Month_str,Month,Month_sin,Month_cos,AdjClosePrev1,AdjClosePrev2,AdjClosePrev3,AdjClosePrev4,AdjClosePrev5,AdjClosePrev6,Days_since
754,2020-01-02,75.150002,73.797501,74.059998,75.087502,135480400.0,74.096443,80.911942,January,1,0.5,0.866025,NaN,NaN,NaN,NaN,NaN,NaN,1064
755,2020-01-03,75.144997,74.125000,74.287498,74.357498,146322800.0,73.376083,80.335754,January,1,0.5,0.866025,74.096443,NaN,NaN,NaN,NaN,NaN,1065
756,2020-01-06,74.989998,73.187500,73.447502,74.949997,118387200.0,73.960770,80.355545,January,1,0.5,0.866025,73.376083,74.096443,NaN,NaN,NaN,NaN,1068
757,2020-01-07,75.224998,74.370003,74.959999,74.597504,108872000.0,73.612930,78.884193,January,1,0.5,0.866025,73.960770,73.376083,74.096443,NaN,NaN,NaN,1069
758,2020-01-08,76.110001,74.290001,74.290001,75.797501,132079200.0,74.797081,80.026649,January,1,0.5,0.866025,73.612930,73.960770,73.376083,74.096443,NaN,NaN,1070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,2020-11-13,119.669998,117.870003,119.440002,119.260002,81581900.0,118.702278,131.352829,November,11,-0.5,0.866025,118.652512,118.931198,115.427666,115.776031,118.134956,118.269310,1380
975,2020-11-16,120.989998,118.150002,118.919998,120.300003,91183000.0,119.737419,136.050766,November,11,-0.5,0.866025,118.702278,118.652512,118.931198,115.427666,115.776031,118.134956,1383
976,2020-11-17,120.669998,118.959999,119.550003,119.389999,74271000.0,118.831673,134.239273,November,11,-0.5,0.866025,119.737419,118.702278,118.652512,118.931198,115.427666,115.776031,1384
977,2020-11-18,119.820000,118.000000,118.610001,118.029999,76322100.0,117.478035,133.094650,November,11,-0.5,0.866025,118.831673,119.737419,118.702278,118.652512,118.931198,115.427666,1385


In [26]:
df_test_lag = df_test_lag.dropna()

In [27]:
df_test_lag

,Date,High,Low,Open,Close,Volume,Adj Close,AdjCloseNextMonth,Month_str,Month,Month_sin,Month_cos,AdjClosePrev1,AdjClosePrev2,AdjClosePrev3,AdjClosePrev4,AdjClosePrev5,AdjClosePrev6,Days_since
760,2020-01-10,78.167503,77.062500,77.650002,77.582497,140644800.0,76.558517,77.412834,January,1,0.5,0.866025,76.385834,74.797081,73.612930,73.960770,73.376083,74.096443,1072
761,2020-01-13,79.267502,77.787498,77.910004,79.239998,121532000.0,78.194145,73.735695,January,1,0.5,0.866025,76.558517,76.385834,74.797081,73.612930,73.960770,73.376083,1075
762,2020-01-14,79.392502,78.042503,79.175003,78.169998,161954400.0,77.138260,71.238098,January,1,0.5,0.866025,78.194145,76.558517,76.385834,74.797081,73.612930,73.960770,1076
763,2020-01-15,78.875000,77.387497,77.962502,77.834999,121923600.0,76.807701,72.368210,January,1,0.5,0.866025,77.138260,78.194145,76.558517,76.385834,74.797081,73.612930,1077
764,2020-01-16,78.925003,78.022499,78.397499,78.809998,108829200.0,77.769821,67.637627,January,1,0.5,0.866025,76.807701,77.138260,78.194145,76.558517,76.385834,74.797081,1078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,2020-11-13,119.669998,117.870003,119.440002,119.260002,81581900.0,118.702278,131.352829,November,11,-0.5,0.866025,118.652512,118.931198,115.427666,115.776031,118.134956,118.269310,1380
975,2020-11-16,120.989998,118.150002,118.919998,120.300003,91183000.0,119.737419,136.050766,November,11,-0.5,0.866025,118.702278,118.652512,118.931198,115.427666,115.776031,118.134956,1383
976,2020-11-17,120.669998,118.959999,119.550003,119.389999,74271000.0,118.831673,134.239273,November,11,-0.5,0.866025,119.737419,118.702278,118.652512,118.931198,115.427666,115.776031,1384
977,2020-11-18,119.820000,118.000000,118.610001,118.029999,76322100.0,117.478035,133.094650,November,11,-0.5,0.866025,118.831673,119.737419,118.702278,118.652512,118.931198,115.427666,1385


In [28]:

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(df_train_lag, df_test_lag, 
        numeric_features+["Month_sin", "Month_cos"]+lag_col+["Days_since"],
        categorical_features+['Month_str'],
        drop_features+['Month'])
lr_pipe = make_pipeline(preprocessor, Ridge())
lr_pipe.fit(df_train_lag, y_train)
print('Training Score:',lr_pipe.score(df_train_lag, y_train))
print('Test Score:',lr_pipe.score(df_test_lag, y_test))

Training Score: 0.8557177825619929
Test Score: 0.6777513336674255
